In [1]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [2]:

internal_urls = set()
external_urls = set()
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [3]:
base_urls = ["https://www.centenaryuniversity.edu/coronavirus-communications/","https://today.citadel.edu/covid-19/","https://www.csm.edu/coronavirus-disease-covid-19"]


In [4]:
for base_url in base_urls:
    internal_urls.add(base_url)
    parsedurl = urlparse(base_url)
    print(parsedurl)
    base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
    print(base_url_main)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        print(count,i)
        if(count == last_count):
            break
        last_count = count
        for immediate_url in immediate_urls:
            try:
                crawl(immediate_url)
                print("[+] NEW Total External links:", len(external_urls))
                print("[+] NEW Total Internal links:", len(internal_urls))
                print("[+] NEW Total:", len(external_urls) + len(internal_urls))
                if len(internal_urls)>1000:
                    break
            except requests.exceptions.ConnectionError:
                continue

    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    
    semifinal_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")


        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)



    for i in semifinal_immediate_urls:
        print(i)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)


    for i in final_immediate_urls:
        print(i)
    total_count = len(final_immediate_urls)
    print(total_count)       
    
    
    
    parent_folder = "Masters" #Change based on folder
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except (urllib.error.HTTPError,urllib.error.URLError) as e:
            print("discarded")
            webpage_cannot_save = webpage_cannot_save+1
            continue

    print(f'Webpages save as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        try:
            if webpage[-4:] == ".pdf":
                url = webpage
                file_name = url.replace(base_url,'')
                file_name = file_name.replace('/','')
                if file_name == '':
                    file_name = "Home"
                pdf_urls.add(webpage)
                print(webpage)
                r = requests.get(url, stream=True)
                with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                    for chunk in r.iter_content(2000):
                        fd.write(chunk)
                pdf_count = pdf_count + 1
        except:
            print("caught")

    print(f"Pdfs saved: {pdf_count}")
    
    
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    
    final_webpage_save_count = webpage_save_count - pdf_count
    print(final_webpage_save_count)
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        try:
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
            webpage_save_counter_loop = webpage_save_counter_loop+1
            print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
        except:
            print("caught")

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        try:
            url = i
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            count = count + 1
            print(f"{count} of {total_count}")
            print(f'Visiting {base_url_text}/{file_name}')
            print(f'...Taking a screenshot')
            driver.get(url)
            S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
            driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
            driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
            print(f'Screenshot of {file_name} page taken! \n')
        except:
            print("caught")
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    driver.quit()

    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

ParseResult(scheme='https', netloc='www.centenaryuniversity.edu', path='/coronavirus-communications/', params='', query='', fragment='')
https://www.centenaryuniversity.edu
www.centenaryuniversity.edu/coronavirus-communications/
www.centenaryuniversity.edu
[INT] Internal link: https://www.centenaryuniversity.edu
[INT] Internal link: https://www.centenaryuniversity.edu/news-media/
[INT] Internal link: https://www.centenaryuniversity.edu/events-calendar/
[INT] Internal link: https://www.centenaryuniversity.edu/tuitionpayment
[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/freshmen/
[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/graduate-students/
[INT] Internal link: https://www.centenaryuniversity.edu/adult-admissions/
[INT] Internal link: https://www.centenaryuniversity.edu/international-programs/
[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/tuition-financial-aid/
[INT] Internal link: https://www.centenaryuniversi

[INT] Internal link: https://www.centenaryuniversity.edu/student-life/class-of-2020/student-involvement/Anju.Thomas@CentenaryUniversity.edu
[INT] Internal link: https://www.centenaryuniversity.edu/student-life/class-of-2020/student-involvement/kelly.munz@centenaryunivertsity.edu
[INT] Internal link: https://www.centenaryuniversity.edu/student-life/class-of-2020/student-involvement/gina.galiano@centenaryuniversity.edu
[INT] Internal link: https://www.centenaryuniversity.edu/student-life/class-of-2020/student-involvement/collaboratory@centenaryuniversity.edu
[INT] Internal link: https://www.centenaryuniversity.edu/academics/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/registrar/registrars-forms/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/registrar/degree-completion/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/registrar/degree-verification/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/registrar/gra

[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/tuition-financial-aid/undergraduate-financial-aid/financial-aid/
[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/tuition-financial-aid/scholarships/outside-scholarships/
[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/tuition-financial-aid/scholarships/awards-for-all-centenary-students/
[INT] Internal link: https://www.centenaryuniversity.edu/admission-aid/tuition-financial-aid/forms-important-dates-and-policies/policies/satisfactory-academic-progress/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/undergraduate-studies/business-administration/business-concentrations/accounting/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/undergraduate-studies/animal-health-equine-pre-vet/
[INT] Internal link: https://www.centenaryuniversity.edu/academics/undergraduate-studies/science-department/animal-health-small-animal-pre-vet-track/
[INT] In

[INT] Internal link: https://www.centenaryuniversity.edu/coronavirus-communications/cares-emergency-financial-grants/helpful-resources-cares/
[INT] Internal link: https://www.centenaryuniversity.edu/coronavirus-communications/cares-emergency-financial-grants/university-resources-cares/
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
9 1
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
[+] NEW Total External links: 310
[+] NEW Total Internal links: 234
[+] NEW Total: 544
[+] NEW Total External links: 342
[+] NEW Total Internal links: 234
[+] NEW Total: 576
[+] NEW Total

[+] NEW Total External links: 370
[+] NEW Total Internal links: 282
[+] NEW Total: 652
[INT] Internal link: https://today.citadel.edu/wp-admin/post.php
[+] NEW Total External links: 396
[+] NEW Total Internal links: 283
[+] NEW Total: 679
2 1
https://today.citadel.edu/covid-19/
https://today.citadel.edu/covid-19/faq/
2
https://today.citadel.edu/covid-19/
https://today.citadel.edu/covid-19/faq/
2
200 can_save - https://today.citadel.edu/covid-19/
200 can_save - https://today.citadel.edu/covid-19/faq/
Webpages save as HTML :2
Webpages unable to save as HTML :0
Pdfs saved: 0
2
2
1 of 2 - https://today.citadel.edu/covid-19/
2 of 2 - https://today.citadel.edu/covid-19/faq/
Saving Completed
1 of 2
Visiting today.citadel.edu/covid-19//Home
...Taking a screenshot
Screenshot of Home page taken! 

2 of 2
Visiting today.citadel.edu/covid-19//faq
...Taking a screenshot
Screenshot of faq page taken! 

Task Completed! Files stored in the Masters\today.citadel.edu_Screenshots Folder
ParseResult(schem